In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv('../input/train.csv')

In [ ]:
train.head()

In [ ]:
train['target'].value_counts() / train.shape[0] * 100  # imbalanced data

# bag of word analysis 

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
wc = WordCloud().generate('i love india; i love bikes; i love apples')
plt.imshow(wc)

In [ ]:
wc = WordCloud().generate(' '.join(train['question_text']))
plt.imshow(wc)

In [ ]:
insincere_rows = train[train['target'] == 1 ]

wc = WordCloud().generate(' '.join(insincere_rows['question_text']))
plt.imshow(wc)

****text cleaning / text transformations 

In [ ]:
# convert all characters to lower case

docs = train['question_text'].str.lower()

In [ ]:
# apply regular expression to retain only alphabets and spaces

docs = docs.str.replace('[^a-z ]', '')
docs.head()

In [ ]:
# remove commonly used words and apply stemming

import nltk
stopwords = nltk.corpus.stopwords.words('english')
custom_stopwords = ['will']
stopwords.extend(custom_stopwords)

In [ ]:
len(stopwords)

In [ ]:
# stemming

stemmer = nltk.stem.PorterStemmer()

In [ ]:
def clean_sentences(text):
    words = text.split(' ')
    clean_words = [stemmer.stem(word) for word in words if word not in stopwords]
    return ' '.join(clean_words)

docs_clean = docs.apply(clean_sentences)
docs_clean.head()

# DOCUMENT TERM MATRIX

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
from sklearn.model_selection import train_test_split

train_model , validate_model = train_test_split(docs_clean , test_size = 0.3 , random_state = 100)

In [ ]:
vectorizer = CountVectorizer(min_df=50)

In [ ]:
vectorizer.fit(train_model)

train_dtm = vectorizer.transform(train_model)
validate_dtm = vectorizer.transform(validate_model)

In [ ]:
train_dtm

In [ ]:
train_model.shape

In [ ]:
unique_terms = vectorizer.get_feature_names()
len(unique_terms)

In [ ]:
unique_terms[:100]

In [ ]:
df_dtm = pd.DataFrame(train_dtm[:10].toarray(),
                     columns = vectorizer.get_feature_names())
df_dtm

In [ ]:
train_x = train_dtm
validate_x = validate_dtm

train_y = train.loc[train_model.index]['target']
validate_y = train.loc[validate_model.index]['target']


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score , f1_score , classification_report

In [ ]:
model_dt = DecisionTreeClassifier(max_depth = 20)
model_dt.fit(train_x , train_y)
validate_pred = model_dt.predict(validate_x)
print(accuracy_score(validate_y,validate_pred))
print(f1_score(validate_y,validate_pred))

# perform same operations on test data 

In [ ]:
test = pd.read_csv('../input/test.csv')
docs_test = test['question_text'].str.lower()
docs_test = docs_test.str.replace('[^a-z ]','')
docs_test_clean = docs_test.apply(clean_sentences)

test_dtm = vectorizer.transform(docs_test_clean)
test_pred = model_dt.predict(test_dtm)

In [ ]:
submission = pd.DataFrame({ 
    'qid' : test['qid'],
    'prediction' : test_pred})

In [ ]:
submission.to_csv('submission.csv',index = False)